In [1]:
%%capture
!unzip -o /content/postgraduate-proj-telescope-img-analysis.zip -d /content/
!mv /content/postgraduate-proj-telescope-img-analysis/* /content/
!rm -rf /content/postgraduate-proj-telescope-img-analysis

In [ ]:
# !wget "https://zenithalbluetechnologiesslu-my.sharepoint.com/:u:/g/personal/onrubia_zenithalblue_com/EZ3e2xrg3dlEqoBbzN03BdIBwyv3Chlca1pN4DmN6r8Osg?download=1" -O data.zip
!wget "https://zenithalbluetechnologiesslu-my.sharepoint.com/:u:/g/personal/onrubia_zenithalblue_com/EXw8YvCd10FHiYFw_gcUR9wBtKjCyK3nFQ035d5krDRhxQ?download=1" -O data.zip

--2025-06-24 15:34:55--  https://zenithalbluetechnologiesslu-my.sharepoint.com/:u:/g/personal/onrubia_zenithalblue_com/EZ3e2xrg3dlEqoBbzN03BdIBwyv3Chlca1pN4DmN6r8Osg?download=1
Resolving zenithalbluetechnologiesslu-my.sharepoint.com (zenithalbluetechnologiesslu-my.sharepoint.com)... 52.107.238.0, 52.107.238.1, 52.107.238.2, ...
Connecting to zenithalbluetechnologiesslu-my.sharepoint.com (zenithalbluetechnologiesslu-my.sharepoint.com)|52.107.238.0|:443... connected.
HTTP request sent, awaiting response... 302 
Location: /personal/onrubia_zenithalblue_com/Documents/Personal/UPC/JOData/images1000.zip?ga=1 [following]
--2025-06-24 15:34:56--  https://zenithalbluetechnologiesslu-my.sharepoint.com/personal/onrubia_zenithalblue_com/Documents/Personal/UPC/JOData/images1000.zip?ga=1
Reusing existing connection to zenithalbluetechnologiesslu-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 8497492080 (7.9G) [application/x-zip-compressed]
Saving to: ‘data.zip’

data.z

In [4]:
%%capture
!unzip /content/data.zip -d /content/

In [5]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.6/962.6 kB 63.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [6]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ronrubii (s-majorek-upc-school) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch

In [ ]:
from config.arg_reader import read_arguments
import tempfile
import os
from train.traineval import train_model, inference
from dataset.managedataset import check_and_split
from config.mode import Mode
import torch



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# mode, task, dev = read_arguments()
mode = Mode.TRAIN
task = "NMS"
dev = []

config = {
        "lr": 1e-3,
        "batch_size": 4,
        "epochs": 200,
        "data_path": "images1000_filtered",
        "output_path": "./output",
        "allow_splitting": True,
        "box_detections_per_img": 1000,
        "train_test_split": [0.9, 0.1],  # full dataset split in train + test
        "train_val_split": [0.9, 0.1],   # train dataset split in train + val
        "crop_size": 512,
        "nms_threshold": 0.5,
        # Checkpointing config
        "checkpointing": {
            "enabled": True,
            # Add as many metrics as wanted. Max if improving means increasing, Min if contrary
            "metrics": {
                "map": "max",
                "map_50": "max",
                "best_iou_per_prediction": "max",
                "best_iou_per_gt": "max"
            },
            "save_path": "checkpoints",  
            "save_last": True           
        }
    }


with tempfile.TemporaryDirectory() as tempdir:

    check_and_split(config,temp_dir=tempdir, device=device)
    
    if mode == Mode.TRAIN:
        train_model(config, tempdir, task, dev, device=device)
    elif mode == Mode.INFER:
        inference(config, tempdir, device=device)



/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Listing folders in: ./images1000
Dataset already split into train and test folders.
🔍 Total imágenes encontradas: 701
🔍 Total etiquetas encontradas: 701
🔍 Total etiqueta vacías: 0


training/loss_box_reg,▁
training/loss_classifier,▁
training/loss_objectness,▁
training/loss_rpn_box_reg,▁
training/loss_box_reg,1.04779
training/loss_classifier,0.65816
training/loss_objectness,17.11511
training/loss_rpn_box_reg,1.71632


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`



Epoch 1/15


train | epoch 1: 100%|██████████| 316/316 [03:44<00:00,  1.41it/s]
eval | epoch 1:  89%|████████▊ | 31/35 [00:31<00:03,  1.06it/s]